<head>
<b><center>CIENCIA DE DATOS</center>

<center>TALLER 2</center>
<center>2024-1</center>

<center>Profesor: Gabriel Jara </center><br>
<center>Integrantes: Fernando Salgado, Cristián González</center><br>
</head>

#### Introducción
---
El presente informe tiene como objetivo presentar el desarrollo del taller 2 de la asignatura de Ciencia de Datos, el cual consiste en el desarrollo de un modelo de clasificación para predecir si un estudiante de la Universidad Técnica Federico Santa María (UTFSM) sede Viña del Mar, se titulará o no. Para ello, se cuenta con un dataset anonimizados ingresado en entre los años 2004 y 2009 y su estado de titulación al 2014.

##### Archivos
---
- `taller_2.ipynb`: Jupyter Notebook con el desarrollo del taller.
- `Taller_2_Titulacion_DatosTaller.csv`: Dataset con los datos de los estudiantes.
- `Taller_2_Titulacion_Evaluacion.csv`: Dataset con los datos de evaluación.


#### Preprocesamiento de datos
---
A continuación se realizará el preprocesamiento de los datos, el cual consiste en la limpieza y transformación de los datos para su posterior análisis. Para ello, se realizará la carga de los datos, la eliminación de columnas innecesarias, la eliminación de filas con datos faltantes, la transformación de variables categóricas a numéricas y la normalización de los datos.
Se debió cambiar el nombre de los archivos de origen quitando tilde en las vocales.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np

# Cargar el conjunto de datos principal
try:
    df_train = pd.read_csv('Taller_2_Titulacion_DatosTaller.csv', encoding='latin1', sep=';')
except UnicodeDecodeError:
    df_train = pd.read_csv('Taller_2_Titulacion_DatosTaller.csv', encoding='iso-8859-1', sep=';')

# Cargar el conjunto de datos de evaluación


# Verificar los nombres de las columnas
print("Columnas del conjunto de datos de entrenamiento:")
print(df_train.columns)

# Verificar si 'Rotulo_Titulo' está en las columnas
estado_titulacion_col = 'Rotulo_Titulo'
if (estado_titulacion_col not in df_train.columns) or ('Id' not in df_train.columns):
    raise KeyError(f"La columna '{estado_titulacion_col}' o 'Id' no se encuentra en el DataFrame.")

# Convertir la variable de estado de titulación a binaria
df_train[estado_titulacion_col] = df_train[estado_titulacion_col].map({'SI': 1, 'NO': 0})

# Separar características y variable objetivo
X = df_train.drop(columns=[estado_titulacion_col, 'Id'])
y = df_train[estado_titulacion_col]

print("\nValores nulos en el conjunto de datos de entrenamiento:")
print(df_train.isnull().sum())


# Eliminar columnas con todos los valores faltantes en el conjunto de entrenamiento
X = X.dropna(axis=1, how='all')

# Codificación de variables categóricas antes de la imputación
X = pd.get_dummies(X)

# Imputar valores nulos en el conjunto de entrenamiento
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Escalado de características
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)



Columnas del conjunto de datos de entrenamiento:
Index(['Id', 'MAT_1SEM_PROM', 'FIS_1SEM_PROM', 'ING_1SEM_PROM', 'ACTF_1SEM_A',
       'ACTF_1SEM_R', 'OTRANS_1SEM_A', 'OTRANS_1SEM_R', 'OTRANS_1SEM_PROM',
       'ESP_1SEM_A', 'ESP_1SEM_R', 'ESP_1SEM_PROM', 'INS_1SEM', 'PROM_1SEM',
       'MAT_2SEM_PROM', 'FIS_2SEM_PROM', 'ING_2SEM_PROM', 'ACTF_2SEM_A',
       'ACTF_2SEM_R', 'OTRANS_2SEM_A', 'OTRANS_2SEM_R', 'OTRANS_2SEM_PROM',
       'ESP_2SEM_A', 'ESP_2SEM_R', 'ESP_2SEM_PROM', 'INS_2SEM', 'PROM_2SEM',
       'INSC_AnO', 'PROM_AnO', 'GENERO', 'NOM_CARRERA', 'TIPO_CARRERA',
       'nombre_nacionalidad', 'descripcion_situacion_egreso_postulante',
       'nombre_comuna_EM', 'nombre_provincia_EM', 'nombre_region_EM',
       'nombre_dependencia_EM', 'nombre_grupo_dependencia_EM',
       'descripcion_rama_educacional_EM', 'nombre_secretaria_admision',
       'numero_estado_civil', 'descripcion_trabajo_remunerado',
       'numero_horario_trabajo', 'descripcion_proseguir_estudios',
       'desc

#### Validación de datos
Se eligio el algoritmo GridSearchCV que aplica la estrategia de validacion K-Fold con k=5 (cv), debido a que se tiene un dataset con un tamaño considerable, su velocidad de ejecución y nos facilita fijar distintos parametros. Consideramos usar validacion simple y Leave One Out Cross-Validation (LOOCV), pero se descarto porque el primero no es recomendable para datasets grandes y el segundo es muy costoso computacionalmente.
El codigo se encuentra en la sección de "Modelo de clasificación".

#### Modelos de clasificación
Para realizar la clasificación se evaluaron los modelos de regresión logística y Random Forest, ambos modelos son ampliamente utilizados para problemas de clasificación.
Se utilizó la metrica de accuracy para evaluar los modelos, que mide la proporción de predicciones correctas que el modelo realiza.


##### Iteracion 1
---
Regresión Logística
<ul>Parametros:
<li>max_iter=1000</li>
<li>random_state=181</li>
</ul>
<ul>Variables:
<li>'C': [0.1, 1, 10]</li>
<li>'penalty': ['l2']</li>
</ul>

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Definir el modelo
model = LogisticRegression(max_iter=1000, random_state=181)

# Definir los parámetros para búsqueda exhaustiva
params = {'C': [0.1, 1, 10], 'penalty': ['l2']}

# Búsqueda exhaustiva para encontrar los mejores parámetros
grid = GridSearchCV(estimator=model, param_grid=params, cv=5, scoring='accuracy')
grid.fit(X_scaled, y)

# Mostrar los mejores parámetros y la precisión media
print("Mejores parámetros para Regresión Logística:", grid.best_params_)
print("Precisión media usando los mejores parámetros:", grid.best_score_)

Mejores parámetros para Regresión Logística: {'C': 0.1, 'penalty': 'l2'}
Precisión media usando los mejores parámetros: 0.7034425005216742


##### Iteracion 2
---
Regresión Logística
<ul>Parametros:
<li>max_iter=1000</li>
<li>random_state=42</li>
</ul>
<ul>Variables:
<li>'C': [0.1, 1, 10]</li>
<li>'penalty': ['l1']</li>
<li>'solver': ['liblinear']</li>
</ul>


In [3]:
# Definir el modelo
model = LogisticRegression(max_iter=1000, random_state=42)

# Definir los parámetros para búsqueda exhaustiva
params = {'C': [0.1, 1, 10], 'penalty': ['l1'], 'solver': ['liblinear']}

# Búsqueda exhaustiva para encontrar los mejores parámetros
grid = GridSearchCV(estimator=model, param_grid=params, cv=5, scoring='accuracy')
grid.fit(X_scaled, y)

# Mostrar los mejores parámetros y la precisión media
print("Mejores parámetros para Regresión Logística:", grid.best_params_)
print("Precisión media usando los mejores parámetros:", grid.best_score_)

Mejores parámetros para Regresión Logística: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Precisión media usando los mejores parámetros: 0.7329247800731442


##### Iteracion 3
---
Random Forest
<ul>Parametros:
<li>random_state=823</li>
</ul>
<ul>Variables:
<li>'max_depth': [None, 10, 20, 30]</li>
<li>'max_features': ['sqrt', 'log2']</li>
<li>'criterion':['gini', 'entropy', 'log_loss']</li>
</ul>

In [4]:
# Definir el modelo
model_rf = RandomForestClassifier(random_state=823)

# Definir los parámetros para búsqueda exhaustiva
params_rf = {'max_depth': [None, 10, 20, 30],'max_features': ['sqrt', 'log2'],'criterion':['gini', 'entropy', 'log_loss']}

# Búsqueda exhaustiva para encontrar los mejores parámetros
grid_rf = GridSearchCV(estimator=model_rf, param_grid=params_rf, cv=5, scoring='accuracy')
grid_rf.fit(X_scaled, y)

# Mostrar los mejores parámetros y la precisión media
print("Mejores parámetros para Random Forest:", grid_rf.best_params_)
print("Precisión media usando los mejores parámetros:", grid_rf.best_score_)

Mejores parámetros para Random Forest: {'criterion': 'gini', 'max_depth': 20, 'max_features': 'sqrt'}
Precisión media usando los mejores parámetros: 0.7518044435658353


##### Iteracion 4
---
Random Forest
<ul>Parametros:
<li>random_state=42</li>
</ul>
<ul>Variables:
<li>'n_estimators': [50, 100, 200]</li>
<li>'max_depth': [None, 10, 20, 30]</li>
<li>'max_features': ['sqrt', 'log2']</li>
<li>'criterion':['gini', 'entropy', 'log_loss']</li>
</ul>


In [5]:
# Definir el modelo
model_rf = RandomForestClassifier(random_state=42)

# Definir los parámetros para búsqueda exhaustiva
params_rf = {'n_estimators': [50, 100, 200],'max_depth': [None, 10, 20, 30],'max_features': ['sqrt', 'log2'],'criterion':['gini', 'entropy', 'log_loss']}

# Búsqueda exhaustiva para encontrar los mejores parámetros
grid_rf = GridSearchCV(estimator=model_rf, param_grid=params_rf, cv=5, scoring='accuracy')
grid_rf.fit(X_scaled, y)

# Mostrar los mejores parámetros y la precisión media
print("Mejores parámetros para Random Forest:", grid_rf.best_params_)
print("Precisión media usando los mejores parámetros:", grid_rf.best_score_)

Mejores parámetros para Random Forest: {'criterion': 'gini', 'max_depth': 20, 'max_features': 'sqrt', 'n_estimators': 200}
Precisión media usando los mejores parámetros: 0.7557779534996101


#### Evaluación sobre casos nuevos
---
Se realiza la evaluación del modelo seleccionado sobre un conjunto de datos proporcionados en el archivo Taller_2_Titulacion_Evaluacion.csv.

In [6]:
# Cargar el conjunto de datos de evaluación
try:
    df_eval = pd.read_csv('Taller_2_Titulacion_Evaluacion.csv', encoding='latin1', sep=';')
except UnicodeDecodeError:
    df_eval = pd.read_csv('Taller_2_Titulacion_Evaluacion.csv', encoding='iso-8859-1', sep=';')

# Preprocesamiento del conjunto de datos de evaluación
X_eval = df_eval.drop(columns=['Id'])
X_eval = pd.get_dummies(X_eval)

# Asegurarse de que X_eval tenga las mismas columnas que X
missing_cols = set(X.columns) - set(X_eval.columns)
for col in missing_cols:
    X_eval[col] = 0
X_eval = X_eval[X.columns]

# Imputar valores nulos
X_eval = pd.DataFrame(imputer.transform(X_eval), columns=X_eval.columns)

# Escalar características
X_eval_scaled = pd.DataFrame(scaler.transform(X_eval), columns=X_eval.columns)



C:\Users\borrar\AppData\Local\Temp\ipykernel_12224\72123807.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_eval[col] = 0
C:\Users\borrar\AppData\Local\Temp\ipykernel_12224\72123807.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_eval[col] = 0
C:\Users\borrar\AppData\Local\Temp\ipykernel_12224\72123807.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) i

In [7]:
# Utilizar el modelo con los mejores parámetros obtenidos
best_model = grid_rf.best_estimator_

# Realizar la predicción sobre el conjunto de evaluación
y_pred = best_model.predict(X_eval_scaled)

# Generar archivo de salida
with open('clasificación_título.txt', 'w') as f:
    for id, prediction in zip(df_eval['Id'], y_pred):
        f.write(f"{id},{['NO', 'SI'][prediction]}\n")

print("Predicciones guardadas en clasificación_título.txt")



Predicciones guardadas en clasificación_título.txt
